In [6]:
import pandas as pd

df = pd.read_csv('sample_data.csv')

def get_met_value(exercise):
    # Define a dictionary of MET values for each exercise
    met_values = {'Push-ups': 3.8, 'Bench press': 3.9, 'Dumbbell flys': 2.9,
                  'Incline bench press': 5.0, 'Cable crossovers': 5.5, 'Lat pulldowns': 3.5,
                  'Seated cable rows': 3.5, 'Deadlifts': 6.0, 'Reverse flys': 2.5,
                  'Pull-ups': 8.0, 'Front raises': 2.5, 'Lateral raises': 3.0,
                  'Arnold press': 3.5, 'Shoulder press': 4.5, 'Face pulls': 2.5,
                  'Concentration curls': 2.5, 'Chin-ups': 6.0, 'Barbell curls': 3.0,
                  'Hammer curls': 3.5, 'Preacher curls': 3.5, 'Tricep pushdowns': 2.5,
                  'Close-grip bench press': 3.8, 'Tricep dips': 3.8, 'Tricep kickbacks': 2.5,
                  'Skull crushers': 2.5, 'Squats': 5.0, 'Leg press': 4.0, 'Leg extensions': 3.0,
                  'Step ups': 5.0, 'Step-ups': 5.0, 'Lunges': 3.3, 'Lying leg curls': 2.5, 'Good mornings': 3.0,
                  'Stiff-legged deadlifts': 5.0, 'Glute-ham raises': 6.0,
                  'Standing calf raises': 3.5, 'Seated calf raises': 2.5, 'Donkey calf raises': 5.0,
                  'Jump rope': 8.0, 'Tibialis raises': 2.0, 'Plank': 3.0, 'Crunches': 3.0,
                  'Leg raises': 3.5, 'Russian twists': 3.0, 'Cable woodchops': 4.0,
                  'Glute bridges': 2.5, 'Hip thrusts': 5.5, 'Single-leg deadlifts': 5.0,
                  'Reverse lunges': 3.5, 'Farmer\'s walk': 6.0, 'Wrist curls': 2.0,
                  'Reverse wrist curls': 2.0, 'Grip squeezes': 2.0, 'Plate pinches': 3.0}

    # Look up the MET value for the given exercise name
    met_value = met_values.get(exercise)

    return met_value

# Apply the function to the 'Exercise' column and store the results in a new column called 'METs'
df['METs'] = df['Exercise'].apply(get_met_value)



In [ ]:
def fitness_function(solution):
    # Convert the solution tensor to a list of indices
    indices = solution.numpy().tolist()
    
    # Create a list of exercises
    exercises = tf.gather(df['Exercise'], indices)
    
    # Check constraints
    num_high_intensity = 0
    num_repeats_over_25 = 0
    muscle_groups = []
    last_intensity = None
    
    def check_constraint(exercise):
        nonlocal num_high_intensity, num_repeats_over_25, muscle_groups, last_intensity
        
        intensity = tf.gather(df['Intensity'], tf.where(df['Exercise'] == exercise))[0, 0]
        repeats = tf.random.uniform([], minval=2, maxval=40, dtype=tf.int32)
        muscle_group = tf.gather(df['Muscle Group'], tf.where(df['Exercise'] == exercise))[0, 0]
        
        # Constraint 1: At least four exercises must have the number of repeats over 25 times
        num_repeats_over_25 += tf.where(repeats > 25, 1, 0)
        
        # Constraint 2: Each training plan should have at least two exercises of high intensity
        num_high_intensity += tf.where(intensity == 'High', 1, 0)
        
        # Constraint 3: Each muscle group repeating in dataframe more than once does not have the same exercise
        def check_muscle_group():
            nonlocal muscle_groups
            
            is_repeating = tf.math.reduce_sum(tf.cast(tf.equal(muscle_group, muscle_groups), tf.int32)) > 0
            last_exercise = tf.gather(exercises, tf.math.maximum(0, indices.index(exercise) - 1))
            is_different = tf.gather(df['Exercise'], tf.where((df['Muscle Group'] == muscle_group) & (df['Exercise'] != last_exercise))) != exercise
            return tf.where(tf.logical_not(tf.logical_and(is_repeating, is_different)), 1.0, 0.0)
        tf.autograph.experimental.set_loop_options(shape_invariants=[(muscle_groups, tf.TensorShape([None]))])
        muscle_groups = tf.concat([muscle_groups, [muscle_group]], axis=0)
        return check_muscle_group()
        
        # Constraint 4: If the last exercise in the fitness training plan was of higher intensity, 
        # the next exercise should be of medium or high intensity
        if last_intensity is not None:
            if last_intensity == 'High' and intensity == 'Low':
                return 0.0
        
        # Constraint 5: Number of repeats are less than 40 and greater than 1
        if repeats <= 1 or repeats >= 40:
            return 0.0
        
        last_intensity = intensity
    
    # Use tf.vectorized_map to apply the check_constraint function to each element of the exercises tensor
    constraints = tf.vectorized_map(check_constraint, exercises)
    
    # Calculate the fitness score
    fitness = tf.reduce_sum(constraints)
    return fitness

In [2]:
import numpy as np
from bat_algorithm import BatAlgorithm

# Define objective function
def rastrigin_function(x):
    A = 10
    return A*len(x) + np.sum(x**2 - A*np.cos(2*np.pi*x))

# Set up Bat Algorithm
dimension = 30
lower_bound = -5.12
upper_bound = 5.12
population_size = 20
max_generations = 100
ba = BatAlgorithm(rastrigin_function, dimension, lower_bound, upper_bound, population_size, max_generations)

# Run Bat Algorithm
best_solution, best_fitness = ba.optimize()

# Print results
print(f'Best solution: {best_solution}')
print(f'Best fitness: {best_fitness}')


TypeError: __init__() got an unexpected keyword argument 'num_bats'

In [2]:
import numpy as np
import pandas as pd
from bat_algorithm import BatAlgorithm

# Read the CSV file into a DataFrame
df = pd.read_csv('sample_data.csv')

# Define the fitness function to be optimized
def fitness_function(solution):
    # Create a list of exercises
    exercises = []
    for i in solution:
        exercises.append(df.iloc[i]['Exercise'])
    
    # Check constraints
    num_high_intensity = 0
    num_repeats_over_25 = 0
    muscle_groups = []
    last_intensity = None
    for i in range(len(exercises)):
        exercise = exercises[i]
        intensity = df.loc[df['Exercise'] == exercise, 'Intensity'].iloc[0]
        repeats = np.random.randint(2, 40)
        muscle_group = df.loc[df['Exercise'] == exercise, 'Muscle Group'].iloc[0]
        
        # Constraint 1: At least four exercises must have the number of repeats over 25 times
        if repeats > 25:
            num_repeats_over_25 += 1
        
        # Constraint 2: Each training plan should have at least two exercises of high intensity
        if intensity == 'High':
            num_high_intensity += 1
        
        # Constraint 3: Each muscle group repeating in dataframe more than once does not have the same exercise
        if muscle_group in muscle_groups:
            last_exercise = exercises[i-1]
            if df.loc[(df['Muscle Group'] == muscle_group) & (df['Exercise'] != last_exercise), 'Exercise'].iloc[0] == exercise:
                return 0.0
        else:
            muscle_groups.append(muscle_group)
        
        # Constraint 4: If the last exercise in the fitness training plan was of higher intensity, 
        # the next exercise should be of medium or high intensity
        if last_intensity is not None:
            if last_intensity == 'High' and intensity == 'Low':
                return 0.0
        
        # Constraint 5: Number of repeats are less than 40 and greater than 1
        if repeats <= 1 or repeats >= 40:
            return 0.0
        
        last_intensity = intensity
    
    # Calculate the fitness score
    fitness = 0.0
    if num_repeats_over_25 >= 4:
        fitness += 1.0
    if num_high_intensity >= 2:
        fitness += 1.0
    return fitness

# Set up Bat Algorithm
dimension = 30
lower_bound = -5.12
upper_bound = 5.12
population_size = 20
max_generations = 100
ba = BatAlgorithm(fitness_function, dimension, lower_bound, upper_bound, population_size, max_generations)

# Run the optimization process
best_solution, best_fitness = ba.optimize()

# Print the results
print("Best solution:", best_solution)
print("Best fitness:", best_fitness)


NameError: name 'df' is not defined